To Do:
automatically detect and adjust if cpu or gpu is available


## Step 0. How this works

This is a Jupyter Notebook file, running in google Colab. that means we get to use their could for our processing power, and also get to do some nice things like connect directly to google drive.

Jupyter Notebook files let you put things like this text box in line with your code, and also execute code in various orders. For this, all we want to do is go through the cells, top to bottom, executing each code cell by clicking the play button in the top left corner. it will have a stop button available while executing then a green check when finished. The output of a given cell will be below the cell, and might give some updates on the progress of execution.

There will also be a cell as your are going down where you will have the option to alter some inputs. This mostly amounts to giving the path to your "Rana_Draytonii_ML_Model" folder and the name for your output excel file.

Everything you need to run predictions is contained in my github repositiory, which will be cloned into this notebook in one of the next cells, and the google drive folder "Rana_Draytonii_ML_Model". You can put this folder wherever you want in your drive. It will have files in it as such:

Always/before you run anything:

*   [Folder] Wav_Files_Input: Where you place files you want processed
*   [File] best_audio_model1.pth: My pretrained weights. leave this be
*   [File] labels.csv: Labels for the model, also let be
*   [File] AST_Inference: thats this file, just wanted everything in one spot



During running it will create:

*   [Folder] temp_file_storage: used in script, doesnt matter outside of while executing these cells
*   [Folder] ResampledAudio: similar to above
*   [File] whatever_you_name_it.xclx: this is the excel file where your results will be stored after everything executes!

Before running any cells, just make sure you have that "Rana_Draytonii_ML_Model" folder in your google drive, and you have the sign in to that drive. One of the next cells will ask you to sign in and give access to your drive when you run it, which will allow this notebook to access those files directly from your drive. I (Tyler Schwenk) will not have access to your drive, this should all be running on your own account. Also make sure you have moved the files you want reviewed into the "Wav_Files_input" folder within that folder (maybe start with an hour and a half or less length of audio at a time [refer to step 3]).

Once those few things are set up, youre ready to roll. Just scroll down and read the text cells and run the code cells, one at a time.
If you have any questions reach out to me at tylerschwenk1@yahoo.com or ask good ol google.

Happy Frogging



## Step 1. Install and import required packages.

These will install onto this instance of google colab. it will not be installed to your computer, and so when you reload this notebook it will need to be installed again.

In [1]:
import sys
from torch.utils.data import Dataset, DataLoader
if 'google.colab' in sys.modules:
    print('Running in Colab.')
    !git clone https://github.com/Tyler-Schwenk/ast-Rana-Draytonii.git
    sys.path.append('./ast-Rana-Draytonii')
%cd /content/ast-Rana-Draytonii/

! pip install timm==0.4.5
! pip install wget
! pip install pydub
import os, csv, argparse, wget
os.environ['TORCH_HOME'] = '/content/ast-Rana-Draytonii/pretrained_models'
if os.path.exists('/content/ast-Rana-Draytonii/pretrained_models') == False:
  os.mkdir('/content/ast-Rana-Draytonii/pretrained_models')
import torch, torchaudio, timm
import numpy as np
import pandas as pd
from torch.cuda.amp import autocast
import IPython
import shutil
import json
import csv
import re
from pydub import AudioSegment
from src.models import ASTModel
import json
from pydub.utils import mediainfo # for retrieving metadata


Running in Colab.
Cloning into 'ast-Rana-Draytonii'...
remote: Enumerating objects: 789, done.
remote: Counting objects: 100% (286/286), done.
remote: Compressing objects: 100% (183/183), done.
remote: Total 789 (delta 141), reused 188 (delta 98), pack-reused 503
Receiving objects: 100% (789/789), 2.48 MiB | 3.06 MiB/s, done.
Resolving deltas: 100% (401/401), done.
/content/ast-Rana-Draytonii
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.4/287.4 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9657 sha256=ee5b90f12bd7187660092ea435a62855b208db9a891fde6e49a735e7da147368
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


/usr/local/lib/python3.10/dist-packages/torch/amp/autocast_mode.py:204: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


**Mount google drive where weights are stored**
 (requires authentication)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Preprocess audio files**

First split into 10 second segments

In [14]:
import math

def split_all_audio_files(input_dir, output_dir, segment_length_sec=10):
    # Make sure output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Loop over all files in the input directory
    for filename in os.listdir(input_dir):
        # Check if the file is a .wav file
        if filename.endswith('.wav') or filename.endswith('.WAV'):
            try:
                # Full path to the original audio file
                audio_path = os.path.join(input_dir, filename)
                # Load the audio file
                waveform, sample_rate = torchaudio.load(audio_path)

                # Calculate number of samples in segment_length_sec
                num_samples_segment = segment_length_sec * sample_rate

                # Calculate total number of segments
                total_segments = math.ceil(waveform.shape[1] / num_samples_segment)

                # Split waveform into segments and save each segment to a new .wav file
                for i in range(total_segments):
                    start = i * num_samples_segment
                    end = start + num_samples_segment
                    segment = waveform[:, start:end]

                    # Prepare filename for the segment
                    segment_filename = f"{filename.rstrip('.wav')}_segment{i}.wav"
                    segment_path = os.path.join(output_dir, segment_filename)

                    # Save segment as a .wav file
                    segment = (segment * 32767).short()  # Convert to 16-bit PCM format
                    torchaudio.save(segment_path, segment, sample_rate)
            except Exception as e:
                  print(f"Error processing file {audio_path}: {str(e)}")
    print("Finished Splitting files into 10 sec segments")

def extract_metadata_from_files_in_directory(directory):
    metadata_dict = {}
    for file in os.listdir(directory):
        if file.endswith(".wav") or file.endswith('.WAV'):  # assuming you're working with wav files
            filepath = os.path.join(directory, file)
            metadata = mediainfo(filepath)

            # Extract desired information from the comment
            comment = metadata.get('TAG', {}).get('comment', '')
            match = re.search(r"Recorded at (.*) by (AudioMoth .*) at .* while .* and temperature was (.*C)\.", comment)
            if match:
                recorded_at, device_id, temperature = match.groups()

                # Create a new dictionary with only the desired information
                simplified_metadata = {
                    'filename': file,
                    'recorded_at': recorded_at,
                    'device_id': device_id,
                    'temperature': temperature,
                }
                metadata_dict[file] = simplified_metadata  # Store metadata using filename as key
    return metadata_dict

## Step 2. Enter Parameters.

In [15]:
#do NOT change variable names (left of equal sign), do change variable contents (red, in quotes) for example:
do_NOT_change = "yes, feel free to change"

model_name = "AST_Rana_Draytonii"
model_version = "1.0"

# output will be stored in this excel file. Definitly change this one
output_file_name = "results.xlsx"
# !! If there is already a file with this name at this location it will be overwritten with the new file !!

# Define the base path to where your Rana7 folder is stored in google drive
# If youre not sure you can find it in the file explorer to the left. /content will always be the base
base_path = '/content/drive/MyDrive/Rana_Draytonii/Rana_Draytonii_ML_Model'

# This is the name of the folder within Rana_Draytionii_ML_Model where you will place the files you want reviewed
input_folder = 'Wav_Files_Input'


**That was all you need to do for inputting data**, now just run the rest of the cells top to bottom

In [16]:
# Defines paths for other files/directories by concatenating with the base path
labels_path = os.path.join(base_path, 'labels.csv')
checkpoint_path = os.path.join(base_path, 'best_audio_model1.pth')
input_dir = os.path.join(base_path, input_folder)
output_dir = os.path.join(base_path, 'Temp_File_Storage')
results_path = os.path.join(base_path, 'results.xlsx')
resampled_audio_dir = os.path.join(base_path, 'ResampledAudio')
metadata_dict = extract_metadata_from_files_in_directory(input_dir)

# Delete all files in the directory
for filename in os.listdir(output_dir):
    file_path = os.path.join(output_dir, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))

print("Temp_File_Storage cleared...")

split_all_audio_files(input_dir, output_dir, segment_length_sec=10)

Temp_File_Storage cleared...
Finished Splitting files into 10 sec segments


Resamples to 16 kHz and converts to mono for use with model:

In [17]:
print("Starting...")

# Delete all files in the directory
for filename in os.listdir(resampled_audio_dir):
    file_path = os.path.join(resampled_audio_dir, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))

print("ResampledAudio cleared...")

# Changes sampling frequency of audio file to 16kHz required by the AST model
def resampler(audio_path, save_dir):
    # load your audio file
    waveform, sample_rate = torchaudio.load(audio_path)

    # define resampler
    resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)

    # resample the waveform
    waveform_resampled = resampler(waveform)

    # create new path with original filename
    base_filename = os.path.basename(audio_path)
    new_path = os.path.join(save_dir, base_filename)

    # save the resampled audio
    torchaudio.save(new_path, waveform_resampled, sample_rate=16000)

    return new_path


# Function to convert strero files to mono
def stereo_to_mono(directory_path):
  # Loop through all files in the directory
  for filename in os.listdir(directory_path):
      # Check if the file is a .wav file
      if filename.endswith('.wav'):
          # Get the full path of the file
          file_path = os.path.join(directory_path, filename)

          # Load audio file
          audio = AudioSegment.from_wav(file_path)

          # If the audio file is stereo
          if audio.channels == 2:
              print(f"Converting stereo file: {filename}")

              # Convert to mono
              mono_audio = audio.set_channels(1)

              # Replace the original file with the mono version
              mono_audio.export(file_path, format='wav')

# Define the target length for your spectrograms (only used for mel spectrogram)
target_length = 1000
mel_bins = 128  # Number of bins in Mel spectrogram

# Convert to mono
stereo_to_mono(output_dir)
print("Files converted to Mono...")

# Process audio samples
for filename in os.listdir(output_dir):
    if filename.endswith('.wav') or filename.endswith('.WAV'):
        filepath = os.path.join(output_dir, filename)
        filepath = resampler(filepath, resampled_audio_dir)  # Resample and get new file path
print("Finished Preprocessing")

Starting...
ResampledAudio cleared...
Converting stereo file: FAUNO-POND_20210318_223500_segment0.wav
Converting stereo file: FAUNO-POND_20210318_223500_segment1.wav
Converting stereo file: FAUNO-POND_20210318_223500_segment2.wav
Converting stereo file: FAUNO-POND_20210318_223500_segment3.wav
Converting stereo file: FAUNO-POND_20210318_223500_segment4.wav
Converting stereo file: FAUNO-POND_20210318_223500_segment5.wav
Files converted to Mono...
Finished Preprocessing


## Step 3. The Big Kahuna

This one will probably take a while.

It will take each file and run it through the ML model to create its prediction of whether or not there is Rana Draytonii.

It give its result as two predictions, the percentage likelyhood that there is a call, and the likelyhood there isnt a call. these probably wont exactly add up to 100% but will be close. the greater prediction is taken as the final prediction.

It will then store the results in an excell file with the Model Name and Version, Filename, Positive score, Negative score, Prediction (positive = frog call; negative = no call), Device ID, Timestamp, and Temperature at recording.

google colab has a limit for how long it can run. "if user does not interact with his Google Colab notebook for more than 90 minutes, its instance is automatically terminated. Also, maximum lifetime of a Colab instance is 12 hours."

so far it seems to be able to process files at about 60-80% of the time of the audio files themselves. that just an estimate from my tests but that would mean you could put an hour and a half worth of .wav files in and walk away and it should finish with no problem before timing out. Thi is also assuming your computer doesnt go to sleep(maybe make sure the sleep time on your computer is set to never before walking away from a big batch).

In the future this could probably be improved by running this Jupyter Notebook in a software like Anaconda instead of Google Colab, which would run it on your personal machine rather than googles cloud. This could give more power (also allowing the script to leverage gpu or multiple cpu cores in order to run faster) but comes at the cost of more memory usage on your machine.

In [18]:
print("Starting...")
from datetime import datetime
from collections import defaultdict
import itertools
class RanaDraytoniiDataset(Dataset):
    def __init__(self, dir_path, transform=None):
        self.transform = transform

        # List all files in the directory
        self.files = [os.path.join(dir_path, f) for f in os.listdir(dir_path) if os.path.isfile(os.path.join(dir_path, f))]

        print(f"Found {len(self.files)} segments")

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        audio_file = self.files[idx]

        if self.transform:
            audio_data = self.transform(audio_file)
            return audio_data
        else:
            return audio_file

def make_features(waveform, mel_bins, target_length=1000):
    sr = 16000  # the sample rate is always 16kHz

    fbank = torchaudio.compliance.kaldi.fbank(
        waveform, htk_compat=True, sample_frequency=sr, use_energy=False,
        window_type='hanning', num_mel_bins=mel_bins, dither=0.0, frame_shift=10)

    n_frames = fbank.shape[0]

    p = target_length - n_frames
    if p > 0:
        m = torch.nn.ZeroPad2d((0, 0, 0, p))
        fbank = m(fbank)
    elif p < 0:
        fbank = fbank[0:target_length, :]

    fbank = (fbank - (-4.2677393)) / (4.5689974 * 2)
    return fbank

def make_features_fixed(wav_name):
    waveform, sr = torchaudio.load(wav_name)
    return make_features(waveform, mel_bins=128)

def load_label(label_csv):
    with open(label_csv, 'r') as f:
        reader = csv.reader(f, delimiter=',')
        next(reader, None)  # skip the headers
        labels = {line[0]: line[1] for line in reader}
    return labels

# Load your labels
labels = load_label(labels_path)

# Load the pre-trained model
model = ASTModel(
    label_dim=2,
    fstride=10,
    tstride=10,
    input_fdim=128,
    input_tdim=1000,
    imagenet_pretrain=False,
    audioset_pretrain=False,
    model_size='base384'
)

# Load your fine-tuned weights
print(f'[*INFO] load checkpoint: {checkpoint_path}')
checkpoint = torch.load(checkpoint_path, map_location='cpu')
audio_model = torch.nn.DataParallel(model, device_ids=[0])
audio_model.load_state_dict(checkpoint)
audio_model = audio_model.to(torch.device("cpu"))
audio_model.eval()
print("Model Loaded...")

# Create a Dataset for your audio files
audio_files_dataset = RanaDraytoniiDataset(resampled_audio_dir, transform=make_features_fixed)

# Create a DataLoader for your audio files
audio_files_data_loader = DataLoader(audio_files_dataset, batch_size=32, shuffle=False)

# Create an empty DataFrame to store the results
results_df = pd.DataFrame(columns=['Model Name : Version', 'File Name', 'Prediction', 'Times Heard', 'Device ID', 'Timestamp', 'Temperature', 'Review Date'])
print('Using model ' + model_name + ' version ' + model_version + ' to make predictions on ' + str(len(audio_files_dataset)) + ' audio files')
print("---------------Start Predictions---------------")

# Add a variable to keep track of the total number of processed files
total_processed_files = 0

metadata_dict = {md['filename']: md for md in metadata_dict.values()}

# Initialize a dictionary to store the predictions for each original audio file
file_predictions = defaultdict(list)

# Iterate over the audio files
for i, batch in enumerate(audio_files_data_loader):
    # Only apply unsqueeze(1) if waveform is 2D
    if len(batch.shape) == 2:
        batch = batch.unsqueeze(1)

    # Move the data to the correct device
    feats_data = batch.to(torch.device("cpu"))

    # Make the prediction
    with torch.no_grad():
        output = audio_model.forward(feats_data)
        output = torch.sigmoid(output)
    result_output = output.data.cpu().numpy()

    # Check each segment for frog call
    for j, file_result in enumerate(result_output):
        try:
            # Get the file name using the total number of processed files
            file_name = os.path.basename(audio_files_dataset.files[total_processed_files + j])

            # Remove the segment part and the extension from the file name
            base_file_name, _ = os.path.splitext(file_name.split('_segment')[0])

            # Get the scores
            positive_score = file_result[0]
            negative_score = file_result[1]

            # Get the prediction
            if positive_score > negative_score:
                file_predictions[base_file_name].append("Positive")
            else:
                file_predictions[base_file_name].append("Negative")

        except Exception as e:
            print(f"Error processing file {audio_files_dataset.files[total_processed_files + j]}: {e}")


    # Update the total number of processed files
    total_processed_files += len(batch)
    print(f"Processed {total_processed_files}/{str(len(audio_files_dataset))} files...")
print("---------------Finished Predictions---------------")

# Group the segments into consecutive ranges
def group_consecutive_elements(data):
    ranges = []
    for k, g in itertools.groupby(enumerate(data), lambda ix: ix[0] - ix[1]):
        consecutive_elements = list(map(lambda x: x[1], g))
        ranges.append((consecutive_elements[0], consecutive_elements[-1]))
    return ranges

# Iterate over the predictions and store the results
for base_file_name, predictions in file_predictions.items():
    heard_segments = [i for i, pred in enumerate(predictions) if pred == "Positive"]

    if len(heard_segments) == 0:
        times_str = 'N/A'
    else:
        heard_ranges = group_consecutive_elements(heard_segments)
        times_str = ', '.join(f'{s*10}-{(e+1)*10}' for s, e in heard_ranges)

    # Get the prediction
    if times_str == 'N/A':
        prediction = 'Negative'
    else:
        prediction = 'Positive!'

    # Try to get metadata for both .WAV and .wav versions of the file
    metadata = metadata_dict.get(base_file_name + '.WAV', metadata_dict.get(base_file_name + '.wav', {}))
    device_id = metadata.get('device_id')
    recorded_at = metadata.get('recorded_at')
    temperature = metadata.get('temperature')

    new_row = pd.DataFrame({
        'Model Name : Version' : [model_name + ':' + model_version],
        'File Name': [base_file_name],
        'Prediction': [prediction],
        'Times Heard': [times_str],  # Add this column
        'Device ID': [device_id],
        'Timestamp': [recorded_at],
        'Temperature': [temperature],
        'Review Date': [datetime.now().strftime('%Y-%m-%d')]  # Add current date
    })

    results_df = pd.concat([results_df, new_row], ignore_index=True)

# Save the DataFrame to an Excel file
results_df.to_excel(results_path, index=False)




Starting...
---------------AST Model Summary---------------
ImageNet pretraining: False, AudioSet pretraining: False
frequncey stride=10, time stride=10
number of patches=1188
[*INFO] load checkpoint: /content/drive/MyDrive/Rana_Draytonii/Rana_Draytonii_ML_Model/best_audio_model1.pth
Model Loaded...
Found 30 segments
Using model AST_Rana_Draytonii version 1.0 to make predictions on 30 audio files
---------------Start Predictions---------------
Processed 30/30 files...
---------------Finished Predictions---------------
